[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/y-akbal/Some-Usefull-Machine-Learning-Stuff/blob/main/HW/HW4.ipynb)


In [1]:
!pip install -U datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 8.9 MB/s eta 0:00:00


In [2]:
 import tensorflow as tf
 from datasets import load_dataset 
 emotions = load_dataset("emotion")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
### This is a dictionary as
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
### We will use training set for the whole stuff
### We will use validation set for fine tuning,
### We will use test set to see if we are doing good!!!!
### Let's get started
### Here is an example of such a tweet
### Each time you run this cell you will get different tweet and a label.
### Please run this cell a couple of times to see some tweets
import numpy as np
index = np.random.randint(0, 1000)
tweet = emotions["train"]["text"][index]
label = emotions["train"]["label"][index]
label_dict = {0:"Saddness", 1: "Joy", 3:"Anger", 4:"Fear", 5:"Surprise", 2:"Love"}
print(f"Tweet: {tweet} \nLabel: {label_dict[label]}")

Tweet: im certainly not going to sit and tell you whats going on in my personal life but i feel that if you were ever curious about whats going in my life all youd have to do is watch the show 
Label: Surprise


## The following cell is a graded cell!!!

In [5]:
### Here let's get some information about the tweets (This is your job BTW)
### 1) Get the lengths and their distribution of the tweets
### 2) Find the length of the longest tweet (remember to use .split() function to tokenize your tweets to find their lengths)
### 2.5) Get the vocab_size
### 3) Check if the labels are uniformly distributed (or see if the data is balanced!!!)
### Your code here!!!!

In [6]:
### We will use some utilities of Tensorflow instead of hardcoding everything as we did in the lecture,
### Remember that we are doing tweet classification
### We won't be using any manual text-to-number functionalities insted we shall use 
# tf.keras.layers.TextVectorization layer

In [7]:
from tensorflow.keras.layers import TextVectorization, LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential

In [8]:
layer = TextVectorization()
## see now https://www.tensorflow.org/guide/keras/preprocessing_layers

In [9]:
X_train, y_train = emotions["train"]["text"], emotions["train"]["label"]
X_val, y_val = emotions["validation"]["text"], emotions["validation"]["label"]
X_test, y_test = emotions["test"]["text"], emotions["test"]["label"]

In [38]:
text_to_num = TextVectorization() ### see properties of this layer as they are pretty important
text_to_num.adapt(X_train) ### this dude automatically fits preprocessing layer

In [39]:
## you can use this layer to feed directly text to your model as
text_to_num(X_train[:10])

<tf.Tensor: shape=(10, 23), dtype=int64, numpy=
number([[    2,   140,     3,   686,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,    40,   102,    60,     9,    15,   500,     5,    15,
         4011,   561,    32,    60,    61,   128,   148,    76,  1529,
            4,    22,  1289,     0,     0],
       [   17,  3316,     7,  1160,     5,   289,     2,     3,   501,
          438,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,    24,   165,     9,   672,    27,     6,  4900,     2,
           59,    47,     8,    13,    22,    72,    30,     6,  3709,
            0,     0,     0,     0,     0],
       [    2,    24,     9,  1087,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [   73,    48,

In [ ]:
#### your code here -- get the vocab size from text_to_num object.
dir(text_to_num)

In [30]:
## Use sequential model and add first text_to_num layer, then embedding layer and then LSTM or GRU, then use dense layer. 
# If you do not make really bad choices in your hyperparameters, you will see at least 85% validation accuracy.
model = Sequential()

In [ ]:
## fit your model here!!!
## your code!!!

In [ ]:
model.predict(["Why so serious???"]) ### This will give you class probabilities,
### use argmax function to get class with the highest probability then use dictionary 
label_dict = {0:"Saddness", 1: "Joy", 3:"Anger", 4:"Fear", 5:"Surprise", 2:"Love"}
### To write a function to convert a given tweet to its class
def label_predict(tweet:str) -> str:
  pass

In [ ]:
### Compute your final accuracy on test set.
## Your code here!!!!